SID: 3040752562

In [2]:
import numpy as np

def circular_convolve(a, b):
    """Circular convolution of two vectors."""
    return np.real(np.fft.ifft(np.fft.fft(a) * np.fft.fft(b)))

def simulate(n=2048):
    """Perform one simulation of the experiment."""
    # Base vector names
    base_vector_names = [
        'person', 'dog', 'cat', 'mouse', 'bite', 'flee', 'cause', 'stroke', 'lick',
        'biteagt', 'biteobj', 'fleeagt', 'fleefrom', 'causeantc', 'causecnsq',
        'strokeagt', 'strokeobj', 'lickagt', 'lickobj'
    ]

    # ID vector names
    id_vector_names = [
        'idjane', 'idjohn', 'idfred', 'idspot', 'idfido', 'idrover', 'idfelix', 'idmort'
    ]

    # Generate base vectors
    base_vectors = {}
    for name in base_vector_names:
        base_vectors[name] = np.random.normal(0, 1/np.sqrt(n), n)

    # Generate id vectors
    id_vectors = {}
    for name in id_vector_names:
        id_vectors[name] = np.random.normal(0, 1/np.sqrt(n), n)

    # Token vectors
    token_vectors = {}
    token_vectors['jane'] = base_vectors['person'] + id_vectors['idjane']
    token_vectors['john'] = base_vectors['person'] + id_vectors['idjohn']
    token_vectors['fred'] = base_vectors['person'] + id_vectors['idfred']
    token_vectors['spot'] = base_vectors['dog'] + id_vectors['idspot']
    token_vectors['fido'] = base_vectors['dog'] + id_vectors['idfido']
    token_vectors['rover'] = base_vectors['dog'] + id_vectors['idrover']
    token_vectors['felix'] = base_vectors['cat'] + id_vectors['idfelix']
    token_vectors['mort'] = base_vectors['mouse'] + id_vectors['idmort']

    # Normalize all vectors to unit length
    for key in base_vectors:
        base_vectors[key] /= np.linalg.norm(base_vectors[key])
    for key in id_vectors:
        id_vectors[key] /= np.linalg.norm(id_vectors[key])
    for key in token_vectors:
        token_vectors[key] /= np.linalg.norm(token_vectors[key])

    # Build HRRs for the probe and episodes
    def build_event(action, agent_role, agent, object_role, obj):
        """Build an event (sentence) according to Plate's thesis. """
        return base_vectors[action] + \
               circular_convolve(base_vectors[agent_role], token_vectors[agent]) + \
               circular_convolve(base_vectors[object_role], token_vectors[obj])

    # Probe: Spot bit Jane, causing Jane to flee from Spot
    Pbite = build_event('bite', 'biteagt', 'spot', 'biteobj', 'jane')
    Pflee = build_event('flee', 'fleeagt', 'jane', 'fleefrom', 'spot')
    Pobjects = token_vectors['jane'] + token_vectors['spot']
    P = base_vectors['cause'] + Pobjects + Pbite + Pflee + \
        circular_convolve(base_vectors['causeantc'], Pbite) + \
        circular_convolve(base_vectors['causecnsq'], Pflee)

    # Initialize a dictionary to store episodes
    episodes = {}

    # E1: Fido bit John, causing John to flee from Fido
    E1bite = build_event('bite', 'biteagt', 'fido', 'biteobj', 'john')
    E1flee = build_event('flee', 'fleeagt', 'john', 'fleefrom', 'fido')
    E1objects = token_vectors['john'] + token_vectors['fido']
    E1 = base_vectors['cause'] + E1objects + E1bite + E1flee + \
         circular_convolve(base_vectors['causeantc'], E1bite) + \
         circular_convolve(base_vectors['causecnsq'], E1flee)
    episodes['E1'] = E1

    # E2: Fred bit Rover, causing Rover to flee from Fred (cross-mapping)
    E2bite = build_event('bite', 'biteagt', 'fred', 'biteobj', 'rover')
    E2flee = build_event('flee', 'fleeagt', 'rover', 'fleefrom', 'fred')
    E2objects = token_vectors['fred'] + token_vectors['rover']
    E2 = base_vectors['cause'] + E2objects + E2bite + E2flee + \
         circular_convolve(base_vectors['causeantc'], E2bite) + \
         circular_convolve(base_vectors['causecnsq'], E2flee)
    episodes['E2'] = E2

    # E3: Felix bit Mort, causing Mort to flee from Felix
    E3bite = build_event('bite', 'biteagt', 'felix', 'biteobj', 'mort')
    E3flee = build_event('flee', 'fleeagt', 'mort', 'fleefrom', 'felix')
    E3objects = token_vectors['felix'] + token_vectors['mort']
    E3 = base_vectors['cause'] + E3objects + E3bite + E3flee + \
         circular_convolve(base_vectors['causeantc'], E3bite) + \
         circular_convolve(base_vectors['causecnsq'], E3flee)
    episodes['E3'] = E3

    # E4: Mort bit Felix, causing Felix to flee from Mort
    E4bite = build_event('bite', 'biteagt', 'mort', 'biteobj', 'felix')
    E4flee = build_event('flee', 'fleeagt', 'felix', 'fleefrom', 'mort')
    E4objects = token_vectors['mort'] + token_vectors['felix']
    E4 = base_vectors['cause'] + E4objects + E4bite + E4flee + \
         circular_convolve(base_vectors['causeantc'], E4bite) + \
         circular_convolve(base_vectors['causecnsq'], E4flee)
    episodes['E4'] = E4

    # E5: Rover bit Fred, causing Rover to flee from Fred
    E5bite = build_event('bite', 'biteagt', 'rover', 'biteobj', 'fred')
    E5flee = build_event('flee', 'fleeagt', 'rover', 'fleefrom', 'fred')
    E5objects = token_vectors['rover'] + token_vectors['fred']
    E5 = base_vectors['cause'] + E5objects + E5bite + E5flee + \
         circular_convolve(base_vectors['causeantc'], E5bite) + \
         circular_convolve(base_vectors['causecnsq'], E5flee)
    episodes['E5'] = E5

    # E6: John fled from Fido, causing Fido to bite John
    E6bite = build_event('bite', 'biteagt', 'fido', 'biteobj', 'john')
    E6flee = build_event('flee', 'fleeagt', 'john', 'fleefrom', 'fido')
    E6objects = token_vectors['john'] + token_vectors['fido']
    E6 = base_vectors['cause'] + E6objects + E6bite + E6flee + \
         circular_convolve(base_vectors['causeantc'], E6flee) + \
         circular_convolve(base_vectors['causecnsq'], E6bite)
    episodes['E6'] = E6

    # E7: Mort bit Felix, causing Mort to flee from Felix
    E7bite = build_event('bite', 'biteagt', 'mort', 'biteobj', 'felix')
    E7flee = build_event('flee', 'fleeagt', 'mort', 'fleefrom', 'felix')
    E7objects = token_vectors['mort'] + token_vectors['felix']
    E7 = base_vectors['cause'] + E7objects + E7bite + E7flee + \
         circular_convolve(base_vectors['causeantc'], E7bite) + \
         circular_convolve(base_vectors['causecnsq'], E7flee)
    episodes['E7'] = E7

    # E8: Mort fled from Felix, causing Felix to bite Mort
    E8bite = build_event('bite', 'biteagt', 'felix', 'biteobj', 'mort')
    E8flee = build_event('flee', 'fleeagt', 'mort', 'fleefrom', 'felix')
    E8objects = token_vectors['mort'] + token_vectors['felix']
    E8 = base_vectors['cause'] + E8objects + E8bite + E8flee + \
         circular_convolve(base_vectors['causeantc'], E8flee) + \
         circular_convolve(base_vectors['causecnsq'], E8bite)
    episodes['E8'] = E8

    # E9: Fido bit John, John fled from Fido
    E9bite = build_event('bite', 'biteagt', 'fido', 'biteobj', 'john')
    E9flee = build_event('flee', 'fleeagt', 'john', 'fleefrom', 'fido')
    E9objects = token_vectors['john'] + token_vectors['fido']
    E9 = E9objects + E9bite + E9flee
    episodes['E9'] = E9

    # E10: Fred stroked Rover, causing Rover to lick Fred
    E10stroke = build_event('stroke', 'strokeagt', 'fred', 'strokeobj', 'rover')
    E10lick = build_event('lick', 'lickagt', 'rover', 'lickobj', 'fred')
    E10objects = token_vectors['fred'] + token_vectors['rover']
    E10 = base_vectors['cause'] + E10objects + E10stroke + E10lick + \
          circular_convolve(base_vectors['causeantc'], E10stroke) + \
          circular_convolve(base_vectors['causecnsq'], E10lick)
    episodes['E10'] = E10

    # E11: Fred stroked Rover, Rover licked Fred
    E11stroke = build_event('stroke', 'strokeagt', 'fred', 'strokeobj', 'rover')
    E11lick = build_event('lick', 'lickagt', 'rover', 'lickobj', 'fred')
    E11objects = token_vectors['fred'] + token_vectors['rover']
    E11 = E11objects + E11stroke + E11lick
    episodes['E11'] = E11

    # Compute dot products (similarities) between probe and episodes
    dot_products = {}
    for key in episodes:
        dot_products[key] = np.dot(P, episodes[key])

    return dot_products

def main():
    n_runs = 500  # Number of simulation runs
    episode_keys = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11']
    results = {key: [] for key in episode_keys}

    # Store similarity results for each episode
    for i in range(n_runs):
        dot_products = simulate()
        for key in episode_keys:
            results[key].append(dot_products[key])

    # Compute mean and standard deviation
    print("Aspects of similarity Dot-products")
    print("Episodes in long-term memory: Type Avg Sd")
    for key in episode_keys:
        avg = np.mean(results[key])
        sd = np.std(results[key])
        print(f"{key}: Avg {avg:.2f} Sd {sd:.3f}")

if __name__ == "__main__":
    main()


Aspects of similarity Dot-products
Episodes in long-term memory: Type Avg Sd
E1: Avg 10.01 Sd 0.472
E2: Avg 5.96 Sd 0.455
E3: Avg 5.01 Sd 0.400
E4: Avg 4.99 Sd 0.408
E5: Avg 7.99 Sd 0.463
E6: Avg 6.00 Sd 0.466
E7: Avg 5.01 Sd 0.388
E8: Avg 3.01 Sd 0.390
E9: Avg 4.99 Sd 0.280
E10: Avg 2.01 Sd 0.378
E11: Avg 1.00 Sd 0.256


Dot products between the probe episode P and episodes E1 to E11 vary in a manner reflecting their analogical similarities categorized by Plate. Episodes like E1 share both surface features and relational structures with P (Literal Similarity); thus, they have higher dot products, showing a strong similarity. Episodes such as E2, E3, and E4 represent Analogies which makes dot products moderate since they share relational structures but differ in objects. By contrast, episodes such as E10 and E11 are much less relationally or superficially similar to P (Objects only) and have correspondingly low dot products.

The dot product provides reasonable estimates of analogical similarity, as it captures both the relational structures and the specific role-filler bindings of the episodes. Since the episodes were encoded using Holographic Reduced Representations, various aspects of the similarity are represented, including object attributes, first-order relations, and higher-order relational structures. It is a holistic representation that allows the dot product to capture the degree to which two episodes are analogous in their underlying relational architecture.

These findings are in concert with Plate's discourse on the dimensions of similarity, showing that analogical reasoning is powerfully modeled under the HRR framework. The dot product similarities capture varieties of analogical similarity since episodes sharing more critical aspects, such as higher-order relations with consistent role mappings, have higher dot products. This suggests that the HRR model effectively captures the structural and relational correspondences that are central to human analogical reasoning.